In [1]:
import os, sys, glob
import numpy as np
import pandas as pd
sys.path.append('../')
from data.mm_dataset import load_mmrf

In [2]:
dset = load_mmrf(fold_span = [1])

loading from: /afs/csail.mit.edu/group/clinicalml/users/rahulgk/trvae/data/../mmrf_files/cleaned_mm_fold.pkl


In [3]:
FDIR  = '/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles'
data_files = {}
for fullname in glob.glob(FDIR+'/*.csv'):
    print (fullname)
    fname = os.path.basename(fullname).split('.')[0]
    data_files[fname] = pd.read_csv(fullname, delimiter=',', encoding='latin-1')
print (data_files.keys())

/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles/STAND_ALONE_ADMISSIONS.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles/PER_PATIENT_VISIT.csv


/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (5,9,10,11,12,13,14,15,20,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,44,46,47,48,49,52,54,55,57,58,59,60,61,63,65,67,68,70,72,73,74,75,76,77,80,81,82,84,87,88,89,91,94,95,96,97,98,100,103,104,105,107,110,111,112,114,117,118,119,123,124,127,129,130,135,136,138,139,140,141,142,144,145,146,147,148,149,150,152,153,154,155,156,157,158,159,160,162,163,164,165,166,167,174,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,215,221,222,223,224,226,227,230,231,232,234,235,237,238,239,240,241,242,252,253,262,265,268,271,294,299,311,312,398,400,441,444,446,448,451,453,455,458,460,462,465,467,469,470,471,473,475,477,478,479,480,481,482,483,484,485,486,487,488,489,490,492,495,498,500,502,505,507,509,512,514,516,519,521,523,526,528,530,533,535,537,540,542

/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles/STAND_ALONE_AE.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles/STAND_ALONE_EMERGENCY_DEPT.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles/STAND_ALONE_FAMHX.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles/STAND_ALONE_MEDHX.csv


/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (14,18,50,51,56,57) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles/STAND_ALONE_SURVIVAL.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles/STAND_ALONE_TREATMENT_REGIMEN.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles/STAND_ALONE_TRTRESP.csv
/afs/csail.mit.edu/group/clinicalml/datasets/multiple_myeloma/ia12/CoMMpass_IA12_FlatFiles/PER_PATIENT.csv
dict_keys(['STAND_ALONE_ADMISSIONS', 'PER_PATIENT_VISIT', 'STAND_ALONE_AE', 'STAND_ALONE_EMERGENCY_DEPT', 'STAND_ALONE_FAMHX', 'STAND_ALONE_MEDHX', 'STAND_ALONE_SURVIVAL', 'STAND_ALONE_TREATMENT_REGIMEN', 'STAND_ALONE_TRTRESP', 'PER_PATIENT'])


In [4]:
N = dset[1]['train']['pids'].shape[0]
np.random.seed(0)
shuf = np.random.permutation(N)

p_idx= shuf[3:6]
pids = dset[1]['train']['pids'][p_idx]
print ('outcome time: ',dset[1]['train']['ys_seq'][p_idx,0])
print ('censorship: ',dset[1]['train']['ce'][p_idx,0])
print ('pat. id:',pids)

outcome time:  [36.666666666666664 23.066666666666666 22.7]
censorship:  [0.0 0.0 0.0]
pat. id: ['MMRF_1260' 'MMRF_1016' 'MMRF_1090']


In [5]:
# Reference values for lab measurements
# min/max/scale (calculated by attempting to ensure that max labs lie b/w 5-8)
healthy_mins_max = {
    'cbc_abs_neut':(2., 7.5,1/3.), # abs neutrophil count (3.67, 1.), (2.83, 4.51)
    'chem_albumin':(34, 50,1/8.), # chemical albumin (43.62, 2.77), (41.30, 45.94)
    'chem_bun':(2.5, 7.1,1/5.), #BUN # reference range, (4.8, 1.15)
    'chem_calcium':(2.2, 2.7,2.), #Calcium, (2.45, 0.125)
    'chem_creatinine':(66, 112,1/36.), # creatinine, (83., 24.85), (62.22, 103.77)
    'chem_glucose':(3.9, 6.9,1/5.), # glucose, (4.91, 0.40), (4.58, 5.24)
    'cbc_hemoglobin':(13., 17.,1), # hemoglobin (12.90, 15.64), (8.86, 1.02)
    'chem_ldh':(2.33, 4.67,1/3.), #LDH, (3.5, 0.585)
    'serum_m_protein':(0.1, 1.1, 1), # M protein (<3 g/dL is MGUS, any presence of protein is pathological); am just using the data mean/std for this, (0.85, 1.89)
    'urine_24hr_m_protein':(0.0, 0.1, 1), # Urine M protein 
    'cbc_platelet':(150, 400,1/60.), # platelet count (206.42, 334.57), (270.5, 76.63)
    'chem_totprot':(6, 8,1/6.), # total protein, (7, 0.5)
    'urine_24hr_total_protein':(0, 0.23, 1), # 
    'cbc_wbc':(3, 10,1/4.), # WBC  (5.71, 8.44), (7.07, 1.63)
    'serum_iga':(0.85, 4.99, 1.), # IgA, (2.92, 1.035)
    'serum_igg':(6.10, 16.16,1/10.), # IgG, (11.13, 2.515)
    'serum_igm':(0.35, 2.42,1), #IgM, (1.385, 0.518)
    'serum_lambda':(0.57, 2.63, 1/2.), #serum lambda, (1.6, 0.515)
    'serum_kappa':(.33, 1.94,1/8.), #serum kappa , (1.135, 0.403)
    'serum_beta2_microglobulin':(0.7, 1.80, 1/3.), #serum_beta2_microglobulin,
    'serum_c_reactive_protein':(0.0, 1., 1.) #serum_c_reactive_protein,
}

## Step 1: Sanity check that longitudinal labs have been captured correctly for a few patients
* Plot the patient's cleaned longitudinal data against their raw data in the files

In [6]:
df = data_files['PER_PATIENT_VISIT']
lab_names = ['D_LAB_cbc_abs_neut', 'D_LAB_chem_albumin', 'D_LAB_chem_bun', 'D_LAB_chem_calcium', 'D_LAB_chem_creatinine',
        'D_LAB_chem_glucose', 'D_LAB_cbc_hemoglobin', 'D_LAB_serum_kappa', 'D_LAB_chem_ldh', 'D_LAB_serum_m_protein', 'D_LAB_cbc_platelet',
        'D_LAB_chem_totprot', 'D_LAB_cbc_wbc', 'D_LAB_serum_iga', 'D_LAB_serum_igg', 'D_LAB_serum_igm', 'D_LAB_serum_beta2_microglobulin',
        'D_LAB_serum_lambda', 'D_LAB_urine_24hr_m_protein', 'D_LAB_urine_24hr_total_protein',
        'D_LAB_serum_c_reactive_protein']
df = df[['PUBLIC_ID','VISITDY']+lab_names]
df = df[df['PUBLIC_ID'].isin(pids.ravel())].reset_index(drop=True)
df.rename(columns = dict([(k,k.split('D_LAB_')[1]) for k in df.columns if 'D_LAB' in k]), inplace=True)

In [7]:
IDX= 1
pname= pids[IDX]
pidx = p_idx[IDX]
print ('Patient ',pname,pidx)

p0 = df[df.PUBLIC_ID==pname]
for cval in p0.columns[1:]:
    if cval in healthy_mins_max:
        p0.loc[:,cval] = (p0[cval]-healthy_mins_max[cval][1])*healthy_mins_max[cval][2]
p0.loc[:,'VISITDY'] = p0.VISITDY//30
p0

Patient  MMRF_1016 397


/data/ml2/software/anaconda3/envs/disease_prog/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,PUBLIC_ID,VISITDY,cbc_abs_neut,chem_albumin,chem_bun,chem_calcium,chem_creatinine,chem_glucose,cbc_hemoglobin,serum_kappa,...,chem_totprot,cbc_wbc,serum_iga,serum_igg,serum_igm,serum_beta2_microglobulin,serum_lambda,urine_24hr_m_protein,urine_24hr_total_protein,serum_c_reactive_protein
0,MMRF_1016,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MMRF_1016,0.0,-0.566667,-1.750,0.1508,-0.70,-0.704667,-0.522,-8.816,-0.167750,...,0.100000,-0.400,-4.45,1.337,-1.16,0.373333,1.8600,NaN,NaN,-0.83
2,MMRF_1016,3.0,-0.633333,-1.125,-0.2062,-1.00,-1.294000,-0.566,-8.072,-0.128375,...,-0.266667,-0.600,-4.38,-0.998,-0.90,NaN,-0.6075,NaN,NaN,NaN
3,MMRF_1016,6.0,-1.366667,-0.750,-0.2776,-0.90,-1.073000,-0.379,-7.576,-0.168250,...,-0.250000,-1.325,-4.49,-1.019,-0.79,-0.050000,-0.8465,NaN,NaN,NaN
4,MMRF_1016,8.0,-0.966667,-1.500,-0.2776,-1.30,-1.146667,-0.346,-7.948,-0.134875,...,-0.283333,-0.900,-4.30,-0.953,-0.40,NaN,-0.3245,NaN,NaN,NaN
5,MMRF_1016,11.0,-1.533333,-1.125,-0.2062,-1.05,-1.048444,-0.478,-8.196,-0.082625,...,-0.366667,-1.400,-4.50,-1.091,-0.82,NaN,-0.2520,NaN,NaN,NaN
6,MMRF_1016,15.0,0.000000,-1.000,-0.4204,-0.85,-0.925667,0.193,-7.762,-0.138625,...,-0.266667,-0.450,-4.44,-1.005,-0.74,NaN,0.2630,NaN,NaN,NaN
7,MMRF_1016,17.0,-1.300000,-0.750,-0.1348,-0.95,-0.655556,-0.500,-7.700,-0.161625,...,-0.316667,-1.275,-4.57,-1.035,-0.86,NaN,0.0230,NaN,NaN,NaN
8,MMRF_1016,21.0,-0.800000,-1.375,0.2222,-1.05,-0.336333,-0.324,-8.754,-0.231875,...,-0.300000,-0.900,-4.77,-0.896,-1.70,NaN,2.4340,NaN,NaN,NaN
9,MMRF_1016,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
## print ('Patient ',dset[1]['train']['pids'][pidx])
for fname, val in zip(dset[1]['train']['feature_names'][3:10],dset[1]['train']['b'][pidx,3:10]):
    print (fname,val)

ecog 0.008833252
serum_beta2_microglobulin -0.15915774
PC1 -0.49415797
PC2 -0.27523112
PC3 0.7986549
PC4 0.04691996
PC5 -0.26687378


In [9]:
print ('Patient ',dset[1]['train']['pids'][pidx])
for idx,fname in enumerate(dset[1]['train']['feature_names_x']):
    print (idx, fname,':',)
    X = dset[1]['train']['x'][pidx]
    M = dset[1]['train']['m'][pidx]
    strv = ''
    for t in range(X.shape[0]):
        if M[t,idx] == 1:
            strv +='('+str(t)+','+'%.3f'%X[t,idx]+'), '
    print (strv)

Patient  MMRF_1016
0 cbc_abs_neut :
(0,-0.567), (3,-0.633), (6,-1.367), (8,-0.967), (11,-1.533), (15,0.000), (17,-1.300), (21,-0.800), 
1 chem_albumin :
(0,-1.750), (3,-1.125), (6,-0.750), (8,-1.500), (11,-1.125), (15,-1.000), (17,-0.750), (21,-1.375), 
2 chem_bun :
(0,0.151), (3,-0.206), (6,-0.278), (8,-0.278), (11,-0.206), (15,-0.420), (17,-0.135), (21,0.222), 
3 chem_calcium :
(0,-0.700), (3,-1.000), (6,-0.900), (8,-1.300), (11,-1.050), (15,-0.850), (17,-0.950), (21,-1.050), 
4 chem_creatinine :
(0,-0.705), (3,-1.294), (6,-1.073), (8,-1.147), (11,-1.048), (15,-0.926), (17,-0.656), (21,-0.336), 
5 chem_glucose :
(0,-0.522), (3,-0.566), (6,-0.379), (8,-0.346), (11,-0.478), (15,0.193), (17,-0.500), (21,-0.324), 
6 cbc_hemoglobin :
(0,-8.816), (3,-8.072), (6,-7.576), (8,-7.948), (11,-8.196), (15,-7.762), (17,-7.700), (21,-8.754), 
7 serum_kappa :
(0,-0.168), (3,-0.128), (6,-0.168), (8,-0.135), (11,-0.083), (15,-0.139), (17,-0.162), (21,-0.232), 
8 serum_m_protein :
(0,1.590), (3,-0.840)

## Step 2: Sanity check that longitudinal lab values have been captured correctly for a few patients
* Plot the patient's longitudinal treatments against their raw data in the files

In [10]:
df = data_files['STAND_ALONE_TRTRESP']
df = df[['public_id','line','trtshnm','trtstdy','trtendy']]
df = df[df['public_id'].isin(pids.ravel())].reset_index(drop=True)
df.loc[:,'trtstdy'] = df.trtstdy//30
df.loc[:,'trtendy'] = df.trtendy//30
df

,public_id,line,trtshnm,trtstdy,trtendy
0,MMRF_1016,1,Bor-Cyc-Dex,0,1
1,MMRF_1016,1,Bor-Len-Cyc-Dex,1,1
2,MMRF_1016,1,Bor-Len-Dex,1,3
3,MMRF_1016,1,Bor-Len-Cyc-Dex,3,3
4,MMRF_1016,1,Bor-Cyc-Dex,3,5
5,MMRF_1016,1,Bor-Len-Dex,7,18
6,MMRF_1016,1,Bor-Car-Len-Cyc-Dex,18,22
7,MMRF_1260,1,Bor-Cyc-Dex,0,3
8,MMRF_1260,1,Bor-Dex,4,34
9,MMRF_1090,1,Bor-Thal-Cyc,0,4


In [11]:
IDX= 1
pname= pids[IDX]
pidx = p_idx[IDX]
print ('Patient ',pname,pidx)

Patient  MMRF_1016 397


In [12]:
print ('Patient ',dset[1]['train']['pids'][pidx])
for idx,fname in enumerate(dset[1]['train']['feature_names_a']):
    print (idx, fname,':',)
    X = dset[1]['train']['a'][pidx]
    M = dset[1]['train']['m_a'][pidx]
    strv = ''
    for t in range(X.shape[0]):
        if M[t,idx] == 1:
            strv +='('+str(t)+','+'%.3f'%X[t,idx]+'), '
    print (strv)

Patient  MMRF_1016
0 Bor :
(0,1.000), (1,1.000), (2,1.000), (3,1.000), (4,1.000), (5,1.000), (7,1.000), (8,1.000), (9,1.000), (10,1.000), (11,1.000), (12,1.000), (13,1.000), (14,1.000), (15,1.000), (16,1.000), (17,1.000), (18,1.000), (19,1.000), (20,1.000), (21,1.000), (22,1.000), 
1 Car :
(0,0.000), (1,0.000), (2,0.000), (3,0.000), (4,0.000), (5,0.000), (7,0.000), (8,0.000), (9,0.000), (10,0.000), (11,0.000), (12,0.000), (13,0.000), (14,0.000), (15,0.000), (16,0.000), (17,0.000), (18,1.000), (19,1.000), (20,1.000), (21,1.000), (22,1.000), 
2 Cyc :
(0,1.000), (1,0.000), (2,0.000), (3,1.000), (4,1.000), (5,1.000), (7,0.000), (8,0.000), (9,0.000), (10,0.000), (11,0.000), (12,0.000), (13,0.000), (14,0.000), (15,0.000), (16,0.000), (17,0.000), (18,1.000), (19,1.000), (20,1.000), (21,1.000), (22,1.000), 
3 Dex :
(0,1.000), (1,1.000), (2,1.000), (3,1.000), (4,1.000), (5,1.000), (7,1.000), (8,1.000), (9,1.000), (10,1.000), (11,1.000), (12,1.000), (13,1.000), (14,1.000), (15,1.000), (16,1.000)

## Step 3: Sanity check that outcomes have been correctly captured

In [13]:
df = data_files['PER_PATIENT']
df = df[['PUBLIC_ID','D_PT_lstalive', 'D_PT_lastdy', 'D_PT_deathdy']]
df = df[df['PUBLIC_ID'].isin(pids.ravel().tolist())].reset_index(drop=True)
df.loc[:,'D_PT_lastdy'] = df.D_PT_lastdy/30.
df.loc[:,'D_PT_deathdy'] = df.D_PT_deathdy/30.
df.loc[:,'D_PT_lstalive'] = df.D_PT_lstalive/30.
df

,PUBLIC_ID,D_PT_lstalive,D_PT_lastdy,D_PT_deathdy
0,MMRF_1016,21.566667,23.066667,23.066667
1,MMRF_1260,35.266667,36.666667,36.666667
2,MMRF_1090,17.533333,22.700000,22.700000


In [14]:
pids.ravel()

array(['MMRF_1260', 'MMRF_1016', 'MMRF_1090'], dtype=object)

In [15]:
for IDX in range(3):
    pname= pids[IDX]
    pidx = p_idx[IDX]
    print ('Patient ',dset[1]['train']['pids'][pidx])
    print (dset[1]['train']['ys_seq'][pidx])
    print (dset[1]['train']['ce'][pidx])

Patient  MMRF_1260
[36.666666666666664]
[0.0]
Patient  MMRF_1016
[23.066666666666666]
[0.0]
Patient  MMRF_1090
[22.7]
[0.0]


In [16]:
if not os.path.exists('mmrf_csv'):
    os.mkdir('mmrf_csv')
dset = load_mmrf(fold_span = range(5))

loading from: /afs/csail.mit.edu/group/clinicalml/users/rahulgk/trvae/data/../mmrf_files/cleaned_mm_fold.pkl


In [17]:
# To CSV for running RF surv
def to_csv(dset):
    for fold in dset:
        for tvt in ['train','valid','test']:
            data    = np.concatenate([dset[fold][tvt]['ys_seq'][:,[0]], dset[fold][tvt]['ce'], dset[fold][tvt]['b'], dset[fold][tvt]['x'][:,0,:], dset[fold][tvt]['a'][:,0,:]], axis=-1)
            cols    = np.concatenate([np.array(['outcome']), np.array(['censored']), dset[fold][tvt]['feature_names'], dset[fold][tvt]['feature_names_x'], dset[fold][tvt]['feature_names_a']],axis=0)
            df = pd.DataFrame(data, columns = cols)
            path = os.path.join('./mmrf_csv',str(fold)+'_'+tvt+'.csv')
            print ('Saving ... ',path)
            df.to_csv(path)
to_csv(dset)

Saving ...  ./mmrf_csv/0_train.csv
Saving ...  ./mmrf_csv/0_valid.csv
Saving ...  ./mmrf_csv/0_test.csv
Saving ...  ./mmrf_csv/1_train.csv
Saving ...  ./mmrf_csv/1_valid.csv
Saving ...  ./mmrf_csv/1_test.csv
Saving ...  ./mmrf_csv/2_train.csv
Saving ...  ./mmrf_csv/2_valid.csv
Saving ...  ./mmrf_csv/2_test.csv
Saving ...  ./mmrf_csv/3_train.csv
Saving ...  ./mmrf_csv/3_valid.csv
Saving ...  ./mmrf_csv/3_test.csv
Saving ...  ./mmrf_csv/4_train.csv
Saving ...  ./mmrf_csv/4_valid.csv
Saving ...  ./mmrf_csv/4_test.csv


In [18]:
df

,PUBLIC_ID,D_PT_lstalive,D_PT_lastdy,D_PT_deathdy
0,MMRF_1016,21.566667,23.066667,23.066667
1,MMRF_1260,35.266667,36.666667,36.666667
2,MMRF_1090,17.533333,22.700000,22.700000
